### Sample code - open issues count

In [1]:
import pandas as pd
merge = pd.read_csv('data/merge_bq_repo_name.csv') # merge_bq_repo_name is the repo name I merged 
merge.shape

(16467, 2)

In [2]:
merge.head(10)

,repo_name,company_name
0,000webhost/000webhost-sdk-js,000webhost
1,100dayproject/polymer-web-example,100dayproject
2,18F/18f-cf-release,18F
3,18F/api-program,18F
4,18F/caddy-boshrelease,18F
5,18F/cf-abacus,18F
6,18F/cf-meteor-buildpack,18F
7,18F/cg-cf-release,18F
8,18F/cg-style,18F
9,18F/cg-style-gem,18F


In [3]:
merge_repo = list(merge['repo_name']) # get only repo name from dataset
len(merge_repo)

16467

### Create pickle to save data in every iteration

In [4]:
# -- save picle  
import pickle
filepath = 'issue_time.pkl'

def batch(time_list):
    
    with open(filepath, 'ab') as f:
        pickle.dump(time_list, f)

### Scrape data using github API

In [5]:
import requests
import time
import json


auth = ('ChuxinYang','91e1d3e627061ed6a05265bfe9d32bd547074941') 

open_issue_time = []

for j in range(len(merge_repo)): 

    page = 1
    while True: 
        
        # use github API to scrape data
        get = requests.get('https://api.github.com/repos/'+ str(merge_repo[j]) +'/issues?page='+ str(page) +'&per_page=100/access_tokens',
                      auth = auth)
        get_json = json.loads(get.text)
        time.sleep(1) # rate limit
        
        # test if this repo still exist
        if type(get_json) == dict:
            break
            
        else:
            if len(get_json) == 0: # if there is nothing inside, break out of the loop and look for next repo
                break
 
                
            for i in range(len(get_json)):
                
                repo_tuple = tuple([merge_repo[j]]) # save the repo name first
                repo_tuple += tuple([get_json[i]['user']['login']]) # save the user login name
                repo_tuple += tuple([get_json[i]['created_at']]) # save the created at time
                repo_tuple += tuple([get_json[i]['updated_at']]) # save the updated at time 
                repo_tuple += tuple([get_json[i]['closed_at']]) # save the closed at time

                batch(repo_tuple) # call batch function to save repo_tuple in a pickle file
                    
            page += 1 
    print(j) # print which repo we are scraping

0
1
2
3
4
5


### Open pickle file and load data

In [6]:
import pickle
import pandas as pd
colnames = ['repo_name', 'user_login','created_at','updated_at','closed_at'] # set column names

info = []
with open('issue_time.pkl', 'rb') as fr:
    try:
        while True:
            info.append(pickle.load(fr))
    except EOFError:
        pass
    
repo_info = pd.DataFrame(info, columns=colnames)
repo_info

,repo_name,user_login,created_at,updated_at,closed_at
0,18F/api-program,gbinal,2016-10-05T20:22:24Z,2016-10-05T20:22:24Z,None
1,18F/api-program,gbinal,2016-09-26T18:39:15Z,2016-09-28T16:21:41Z,None
2,18F/api-program,gbinal,2016-09-15T15:02:43Z,2016-09-15T15:02:43Z,None
3,18F/api-program,gbinal,2016-08-18T17:49:00Z,2016-08-18T17:49:09Z,None
4,18F/api-program,gbinal,2016-07-29T12:01:49Z,2016-07-29T12:01:49Z,None
...,...,...,...,...,...
86,18F/api-program,gbinal,2015-02-11T18:46:17Z,2015-04-14T20:10:05Z,None
87,18F/api-program,gbinal,2015-02-11T18:44:24Z,2015-04-14T20:10:13Z,None
88,18F/api-program,gbinal,2015-02-11T18:43:32Z,2015-04-14T20:10:25Z,None
89,18F/api-program,gbinal,2015-02-11T18:40:30Z,2015-04-14T20:11:17Z,None
